# 作业一：实现HMM中文分词和BPE英文分词
姓名：陈永俊

学号：522031910203

## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [441]:
import pickle
import numpy as np

导入HMM参数，初始化所需的起始概率矩阵、转移概率矩阵、发射概率矩阵。

In [442]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的Unicode码点（一个整数值）作为行索引
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子：

In [443]:
# TODO: 将your_name中的xxx替换为你的姓名【1分】
your_name = "陈永俊"
input_sentence = f"{your_name}是一名优秀的学生"

实现Viterbi算法，并以此进行中文分词。

In [444]:
def viterbi(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> str:
    """
    Viterbi算法进行中文分词。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)

    #  TODO: 第一个位置的最大概率值计算【1分】
    pass
    dp[0][0] = start_prob[0]*emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1]*emission_mat[1][sent_ord[0]]
    # argmax 
    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）【2分】
    pass
    # path[i][j] = argmax_{k} dp[i][j]
    # dp[i][j] = max_{k}(dp[k][j-1]*trans_mat[k][i]*emission_mat[i][sent_ord[j]])
    # for i in range(1,len(sent_ord)):
    for j in range(1, len(sent_ord)):
        for i in range(0,2):
            max_prob, max_state = max(
                (dp[k][j-1] * trans_mat[k][i] * emission_mat[i][sent_ord[j]], k) for k in range(2)
            )
            dp[i][j] = max_prob
            path[i][j] = max_state
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]

    #  TODO: 计算labels每个位置上的值（填充labels矩阵）【1分】
    pass
    # 从最后一个位置开始回溯
    labels[-1] = np.argmax(dp[:,-1])
    for j in range(len(sent_ord)-1,0,-1):
        labels[j-1] = path[labels[j]][j]
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str

In [445]:
print("Viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

Viterbi算法分词结果： 陈/永俊是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值。

In [446]:
def compute_prob_by_forward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    前向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算【1分】
    pass
    dp[0][0] = start_prob[0]*emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1]*emission_mat[1][sent_ord[0]]
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    pass
    for i in range (1,len(sent_ord)):
        
            dp[0][i] = sum([dp[k][i-1] * trans_mat[k][0] * emission_mat[0][sent_ord[i]] for k in range(2)])
            dp[1][i] = sum([dp[k][i-1] * trans_mat[k][1] * emission_mat[1][sent_ord[i]] for k in range(2)])
    return sum([dp[i][len(sent_ord) - 1] for i in range(2)])

实现后向算法，计算该句子的概率值。

In [447]:
def compute_prob_by_backward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    后向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化【1分】
    pass
    dp[0][len(sent_ord)-2] = sum([1 * trans_mat[0][i] * emission_mat[i][sent_ord[len(sent_ord)-1]] for i in range(2)])
    dp[1][len(sent_ord)-2] = sum([1 * trans_mat[1][i] * emission_mat[i][sent_ord[len(sent_ord)-1]] for i in range(2)])
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    pass
    for k in range(len(sent_ord)-3,-1,-1):
        dp[0][k] = sum([dp[i][k+1]*trans_mat[0][i]*emission_mat[i][sent_ord[k+1]] for i in range(2)])
        dp[1][k] = sum([dp[i][k+1]*trans_mat[1][i]*emission_mat[i][sent_ord[k+1]] for i in range(2)])
    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [448]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 4.857127292636688e-34
后向算法概率： 4.857127292636689e-34


### 实验总结
> TODO：请在这里填写实验总结。

## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有6处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [449]:
import re
from typing import List, Tuple, Dict

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [450]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")


def white_space_tokenize(corpus: List[str]) -> List[List[str]]:
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到 `[["i", "am", "happy"], ["i", "have", "N", "apples"]]`

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(filter(lambda token: len(token) > 0, _splitor_pattern.split(_digit_pattern.sub("N", sentence.lower())))) for sentence in corpus]

    return tokeneds

corpus = ["I am happy.", "I have 10 apples!"]
print(white_space_tokenize(corpus))

[['i', 'am', 'happy'], ['i', 'have', 'N', 'apples']]


编写相应函数构建BPE算法需要用到的初始状态词典。

In [451]:
def build_bpe_vocab(corpus: List[str]) -> Dict[str, int]:
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到
    ```python
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()

    # TODO: 完成函数体【1分】
    for wordlist in tokenized_corpus:
        for word in wordlist:
            bpe_word = ' '.join(list(word))+' </w>'
            if bpe_word in bpe_vocab:
                bpe_vocab[bpe_word] += 1
            else:
                bpe_vocab[bpe_word] = 1
    return bpe_vocab

corpus = ["I am happy.", "I have 10 apples!"]
print(build_bpe_vocab(corpus))

{'i </w>': 2, 'a m </w>': 1, 'h a p p y </w>': 1, 'h a v e </w>': 1, 'N </w>': 1, 'a p p l e s </w>': 1}


编写所需的其他函数：

In [452]:
def get_bigram_freq(bpe_vocab: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：  
    输入 
    ```python
    bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple[str, str], int] - "bigram->频数"的词典
    """

    bigram_freq = dict()

    # TODO: 完成函数体【1分】
    pass
    for word in bpe_vocab:
        tokens = word.split()  # 将单词按空格分词
        for i in range(len(tokens) - 1):  # 遍历每个 bigram
            bigram = (tokens[i], tokens[i + 1])
            if bigram in bigram_freq:
                bigram_freq[bigram] += bpe_vocab[word]
            else:
                bigram_freq[bigram] = bpe_vocab[word]
    return bigram_freq

bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
print(get_bigram_freq(bpe_vocab))

{('i', '</w>'): 2, ('a', 'm'): 1, ('m', '</w>'): 1, ('h', 'a'): 2, ('a', 'p'): 2, ('p', 'p'): 2, ('p', 'y'): 1, ('y', '</w>'): 1, ('a', 'v'): 1, ('v', 'e'): 1, ('e', '</w>'): 1, ('N', '</w>'): 1, ('p', 'l'): 1, ('l', 'e'): 1, ('e', 's'): 1, ('s', '</w>'): 1}


In [453]:
def refresh_bpe_vocab_by_merging_bigram(bigram: Tuple[str, str], old_bpe_vocab: Dict[str, int]) -> Dict[str, int]:
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：  
    输入 
    ```python
    bigram = ('i', '</w>'), old_bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    
    Args:
        bigram: Tuple[str, str] - 待合并的bigram
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """

    new_bpe_vocab = dict()
    bigram_str = ' '.join(bigram)
    bigram_merged = ''.join(bigram)
    # TODO: 完成函数体【1分】
    pass
    for word,freq in old_bpe_vocab.items():
        if bigram_str in word:
            new_word = word.replace(bigram_str,bigram_merged)
            new_bpe_vocab[new_word] = freq
        else:
            new_bpe_vocab[word] = old_bpe_vocab[word]
    return new_bpe_vocab

bigram = ('h', 'a')

old_bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

print(refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab))

{'i </w>': 2, 'a m </w>': 1, 'ha p p y </w>': 1, 'ha v e </w>': 1, 'N </w>': 1, 'a p p l e s </w>': 1}


In [454]:
def get_bpe_tokens(bpe_vocab: Dict[str, int]) -> List[Tuple[str, int]]:
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：  
    输入 
    ```python
    bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    ```
    得到
    ```
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
    ]
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple[str, int]] - BPE分词和对应频数组成的List
    """

    # TODO: 完成函数体【2分】
    pass
    bpe_tokens = []
    
    from collections import defaultdict
    token_counts = defaultdict(int)

    for word in bpe_vocab:
        tokens = word.split()  # 将单词分词状态按空格拆分为分词
        for token in tokens:
            token_counts[token] += bpe_vocab[word]  # 累加频数
    
    # 将结果转换为列表，并按照分词长度降序排序
    bpe_tokens = sorted(token_counts.items(), key=lambda x: (-len(x[0]), x[0]))

    return bpe_tokens

bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }

get_bpe_tokens(bpe_vocab)

[('i</w>', 2),
 ('</w>', 5),
 ('ha', 2),
 ('pp', 2),
 ('N', 1),
 ('a', 2),
 ('e', 2),
 ('l', 1),
 ('m', 1),
 ('s', 1),
 ('v', 1),
 ('y', 1)]

In [455]:
def print_bpe_tokenize(word: str, bpe_tokens: List[Tuple[str, int]]):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，  
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，  
    或者剩余部分无法匹配（该部分整体由`"<unknown>"`代替）
    
    例1：  
    输入 `word = "supermarket"`, `bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]`  
    最终打印 `"su per <unknown>"`

    例2：  
    输入 `word = "shanghai"`, `bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]`  
    最终打印 `"sh an g hai </w>"`

    Args:
        word: str - 待分词的单词
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的列表
    """

    # TODO: 请尝试使用递归函数定义该分词过程【2分】
    def bpe_tokenize(sub_word: str) -> str:
        # 如果子串为空，直接返回空字符串
        if not sub_word:
            return ""

        # 遍历 BPE 分词，寻找第一个在 sub_word 中出现的 token
        for token, _ in bpe_tokens:
            pos = sub_word.find(token)
            if pos != -1:
                # 找到匹配的分词，递归处理左右部分
                left_part = sub_word[:pos]
                right_part = sub_word[pos + len(token):]
                return (
                    (bpe_tokenize(left_part) + " " if left_part else "")
                    + token + " "
                    + (bpe_tokenize(right_part) if right_part else "")
                ).strip()

        # 如果没有找到匹配的分词，返回 "<unknown>"
        return "<unknown> "

    res = bpe_tokenize(word + "</w>")
    print(res)

# 测试示例
bpe_tokens = [
    ("su", 20),
    ("are", 10),
    ("per", 30),
]
print_bpe_tokenize("supermarket", bpe_tokens)

bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
print_bpe_tokenize("shanghai", bpe_tokens)

su per <unknown>
sh an g hai </w>


开始读取数据集并训练BPE分词器：

In [456]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


In [457]:
training_iter_num = 300

# 构建初始 BPE 词典
training_bpe_vocab = build_bpe_vocab(training_corpus)

for i in range(training_iter_num):
    # 获取当前 BPE 词典中的所有 bigram 及其频数
    bigram_freq = get_bigram_freq(training_bpe_vocab)

    # 检查是否有 bigram 存在，否则停止训练
    if not bigram_freq:
        break
    
    # 找到出现频次最高的 bigram
    most_frequent_bigram = max(bigram_freq, key=bigram_freq.get)
    
    # 根据该 bigram 更新 BPE 词典
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(most_frequent_bigram, training_bpe_vocab)
    # 输出当前迭代信息
    print(f"Iteration {i}: Merged bigram {most_frequent_bigram}")

# 获得最终的 BPE 分词
training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)
print("Final BPE tokens:", training_bpe_tokens)


Iteration 0: Merged bigram ('e', '</w>')
Iteration 1: Merged bigram ('s', '</w>')
Iteration 2: Merged bigram ('t', 'h')
Iteration 3: Merged bigram ('d', '</w>')
Iteration 4: Merged bigram ('i', 'n')
Iteration 5: Merged bigram ('t', '</w>')
Iteration 6: Merged bigram ('e', 'r')
Iteration 7: Merged bigram ('a', 'n')
Iteration 8: Merged bigram ('o', 'n')
Iteration 9: Merged bigram ('th', 'e</w>')
Iteration 10: Merged bigram ('y', '</w>')
Iteration 11: Merged bigram ('e', 'n')
Iteration 12: Merged bigram ('o', 'r')
Iteration 13: Merged bigram ('a', 'r')
Iteration 14: Merged bigram ('o', '</w>')
Iteration 15: Merged bigram ('e', 'd</w>')
Iteration 16: Merged bigram ('a', 'l')
Iteration 17: Merged bigram ('N', '</w>')
Iteration 18: Merged bigram ('g', '</w>')
Iteration 19: Merged bigram ('a', '</w>')
Iteration 20: Merged bigram ('r', 'e')
Iteration 21: Merged bigram ('o', 'u')
Iteration 22: Merged bigram ('in', 'g</w>')
Iteration 23: Merged bigram ('t', 'i')
Iteration 24: Merged bigram ('f',

测试BPE分词器的分词效果：

In [458]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n atur al lan gu age pro ce s sing</w>


### 实验总结
> TODO：请在这里填写实验总结。